<a href="https://colab.research.google.com/github/ameygawade/DataScience-Water_Quality_Pred/blob/main/Water_Quality_Pred.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step-1 ENV setup

In [ ]:
!pip install google-cloud-bigquery pandas scikit-learn

# Step-2 GCP authentication

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated to Google Cloud')

Authenticated to Google Cloud


## Step-3 Initializing BigQuery client

In [ ]:
from google.cloud import bigquery

# IMPORTANT: Replace with your actual Project ID
PROJECT_ID = 'water-quality-param-pred'
client = bigquery.Client(project=PROJECT_ID)

# Step-4 Data Loading

In [ ]:
# 1. SQL to fetch the final data
QUERY = """
SELECT
    *
FROM
    `water-quality-param-pred.water_data_raw.water_quality_features_final`
"""

# 2. Fetch data into a Pandas DataFrame
df = client.query(QUERY).to_dataframe()

print(f"Data retrieved successfully. Total rows: {len(df)}")
print(df.head())

# Continue with the rest of the Day 2 modeling steps (splitting, training, evaluation)

Data retrieved successfully. Total rows: 2171
      measurement_timestamp Site_Id  DO_mg_L  pH_clean  Air_Temp_C  \
0 2004-01-04 00:00:00+00:00       A      8.5       7.0    0.555556   
1 2004-01-04 00:00:00+00:00       B      8.5       7.0    0.555556   
2 1993-01-24 00:03:44+00:00     Bay     10.5       7.0   14.500000   
3 1997-01-05 00:12:20+00:00     Bay      9.2       6.7   17.400000   
4 1998-01-04 00:07:28+00:00     Bay     12.3       6.4   14.000000   

   Water_Temp_C  Salinity  Secchi_Depth_m  day_of_week  month_of_year  \
0           5.0       1.0            0.65            1              1   
1           4.0       1.0            0.20            1              1   
2          10.5       3.3            0.30            1              1   
3          13.5       2.0            0.20            1              1   
4           8.9       1.5            0.25            1              1   

    DO_lag_1  DO_lag_7  Air_Temp_7D_Avg  
0   5.416033  5.416033        17.190476  
1   5.3408

In [ ]:
# check null values
print(df.isnull().sum())

measurement_timestamp    0
Site_Id                  0
DO_mg_L                  0
pH_clean                 0
Air_Temp_C               0
Water_Temp_C             0
Salinity                 0
Secchi_Depth_m           0
day_of_week              0
month_of_year            0
DO_lag_1                 0
DO_lag_7                 0
Air_Temp_7D_Avg          0
dtype: int64


In [ ]:
# group by site_id

df.groupby('Site_Id').count()

,measurement_timestamp,DO_mg_L,pH_clean,Air_Temp_C,Water_Temp_C,Salinity,Secchi_Depth_m,day_of_week,month_of_year,DO_lag_1,DO_lag_7,Air_Temp_7D_Avg
Site_Id,,,,,,,,,,,,
A,402,402,402,402,402,402,402,402,402,402,402,402
B,402,402,402,402,402,402,402,402,402,402,402,402
Bay,739,739,739,739,739,739,739,739,739,739,739,739
C,228,228,228,228,228,228,228,228,228,228,228,228
D,399,399,399,399,399,399,399,399,399,399,399,399


## Step-5 Data Cleaning

In [ ]:
# 1. Impute Salinity
# Calculate the median Salinity for each unique Site_Id
site_salinity_median = df.groupby('Site_Id')['Salinity'].median()

# Fill nulls in Salinity by mapping the Site's median and ASSIGNING the result back to the column
# CORRECTED: Use direct assignment instead of inplace=True
df['Salinity'] = df['Salinity'].fillna(df['Site_Id'].map(site_salinity_median))

print(f"Salinity nulls remaining: {df['Salinity'].isnull().sum()}")


# 2. Impute Secchi Depth
# Calculate the median Secchi Depth for each unique Site_Id
site_secchi_median = df.groupby('Site_Id')['Secchi_Depth_m'].median()

df['Secchi_Depth_m'] = df['Secchi_Depth_m'].fillna(df['Site_Id'].map(site_secchi_median))

print(f"Secchi_Depth_m nulls remaining: {df['Secchi_Depth_m'].isnull().sum()}")


# 3. Impute Do_mg_L
# Calculate the median value for each uniquw Site_Id

site_do_median = df.groupby('Site_Id')['DO_mg_L'].median()

df['DO_mg_L'] = df['DO_mg_L'].fillna(df['Site_Id'].map(site_do_median))

print(f"DO_mg_L nulls remaining: {df['DO_mg_L'].isnull().sum()}")

# 4. Impute Do_lag_1 and DO_lag_7
# Use linear interpoplate for each unique Site_id
df['DO_lag_1'] = df['DO_lag_1'].interpolate(method='linear')
df['DO_lag_7'] = df['DO_lag_7'].interpolate(method='linear')

print(f"DO_lag_1 nulls remaining: {df['DO_lag_1'].isnull().sum()}")
print(f"DO_lag_7 nulls remaining: {df['DO_lag_7'].isnull().sum()}")


Salinity nulls remaining: 0
Secchi_Depth_m nulls remaining: 0
DO_mg_L nulls remaining: 1
DO_lag_1 nulls remaining: 0
DO_lag_7 nulls remaining: 0


In [ ]:
# Check missing value
df[df['DO_mg_L'].isnull()]

,measurement_timestamp,Site_Id,DO_mg_L,pH_clean,Air_Temp_C,Water_Temp_C,Salinity,Secchi_Depth_m,day_of_week,month_of_year,DO_lag_1,DO_lag_7,Air_Temp_7D_Avg
1622,2015-07-23 00:17:35+00:00,d,NaN,7.0,27.777778,30.5,0.0,0.8,5,7,6.199687,6.8,27.777778


In [ ]:
# Drop rows where DO_mg_L is NaN (Not a Number/missing)
df.dropna(subset=['DO_mg_L'], inplace=True)

In [ ]:
# Final verification
final_null_counts = df.isnull().sum()
print("\nFinal Null Count Check (Should be 0):")
print(final_null_counts)

print("\n✅ Data is now fully prepared and clean for modeling!")


Final Null Count Check (Should be 0):
measurement_timestamp    0
Site_Id                  0
DO_mg_L                  0
pH_clean                 0
Air_Temp_C               0
Water_Temp_C             0
Salinity                 0
Secchi_Depth_m           0
day_of_week              0
month_of_year            0
DO_lag_1                 0
DO_lag_7                 0
Air_Temp_7D_Avg          0
dtype: int64

✅ Data is now fully prepared and clean for modeling!


In [ ]:
# Define the feature categories based on your features list:
numerical_features = [
    'DO_lag_1', 'DO_lag_7', 'Air_Temp_C', 'Water_Temp_C', 'Salinity',
    'Secchi_Depth_m', 'Air_Temp_7D_Avg', 'pH_clean'
]

# Use the time features as categorical, as they represent cycles
categorical_features = [
    'day_of_week', 'month_of_year'
]
features = numerical_features + categorical_features # Combine all features for slicing the DataFrame
target = 'DO_mg_L'


# --- IMPORTS for Pipeline ---
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

# --- A. Define Preprocessing Steps ---

# 1. Numerical Preprocessor: Scale all numerical features
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# 2. Categorical Preprocessor: One-hot encode the time cycle features
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# 3. Create a Column Transformer: Applies the correct transformations to the correct columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough' # Keep any other columns (should be none here)
)

# --- B. Create the Full ML Pipeline ---

# We chain the preprocessor with the Gradient Boosting Regressor
gbr_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', GradientBoostingRegressor(
        n_estimators=200,
        max_depth=5,
        learning_rate=0.05,
        random_state=42
    ))
])

# --- C. TRAIN THE PIPELINE ---
print("\nTraining Advanced Pipeline (Gradient Boosting)...")

# 🚨 --- C. DATA SPLIT (THE MISSING STEP) --- 🚨
# Use the last 20% of the data chronologically for testing (Time Series Split)
train_size = int(len(df) * 0.8)

# Define X (all features) and Y (target) for splitting
X = df[features]
y = df[target]

# Split the data
X_train = X.iloc[:train_size]
X_test = X.iloc[train_size:]
y_train = y.iloc[:train_size]
y_test = y.iloc[train_size:]


gbr_pipeline.fit(X_train, y_train)
y_pred_gbr = gbr_pipeline.predict(X_test)

# --- D. EVALUATION (Using the fixed RMSE calculation) ---
rmse_gbr = np.sqrt(mean_squared_error(y_test, y_pred_gbr))
r2_gbr = r2_score(y_test, y_pred_gbr)

print("\n--- Advanced Model Results (Pipeline) ---")
print(f"Root Mean Squared Error (RMSE): {rmse_gbr:.3f}")
print(f"R-squared (R²): {r2_gbr:.3f}")
print("---------------------------------------")



Training Advanced Pipeline (Gradient Boosting)...

--- Advanced Model Results (Pipeline) ---
Root Mean Squared Error (RMSE): 1.305
R-squared (R²): 0.531
---------------------------------------


In [ ]:
import pandas as pd

# --- FEATURE IMPORTANCE ANALYSIS with Pipeline ---
print("\n--- Feature Importances (Pipeline Interpretation) ---")

# 1. Get the final estimator (GradientBoostingRegressor)
final_estimator = gbr_pipeline.named_steps['regressor']

# 2. Get the feature importances from the estimator
importances = final_estimator.feature_importances_

# 3. Get the feature names from the ColumnTransformer
# Get names of one-hot encoded features
# NOTE: This assumes the numerical_features and categorical_features lists from the previous step are in scope.
one_hot_features = list(gbr_pipeline.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_features))

# Combine the numerical names with the one-hot encoded names
feature_names_out = numerical_features + one_hot_features

# 4. Create the importance series
feature_importance_pipe = pd.Series(importances, index=feature_names_out).sort_values(ascending=False)

# Display the top 5 most critical predictors
print("Top 5 Predictors of Dissolved Oxygen (DO_mg_L):")
print(feature_importance_pipe.head(5))


--- Feature Importances (Pipeline Interpretation) ---
Top 5 Predictors of Dissolved Oxygen (DO_mg_L):
DO_lag_1           0.455041
Water_Temp_C       0.170514
DO_lag_7           0.134700
Air_Temp_7D_Avg    0.051313
Secchi_Depth_m     0.049811
dtype: float64


In [ ]:
# --- 5. PREPARE AND UPLOAD RESULTS TO BIGQUERY ---

# 1. Append predictions to the test set data for upload
# 'results_df' is the copy of the test set rows from the original DataFrame.
results_df = df.iloc[train_size:].copy()
results_df['DO_predicted'] = y_pred_gbr

# 2. Define 'results_to_upload'
# This selects ONLY the columns needed for the Looker Studio visualization.
results_to_upload = results_df[[
    'measurement_timestamp', 'Site_Id', target, 'DO_predicted',
    'Air_Temp_C', 'Water_Temp_C', 'Salinity'
]]

# 3. Upload 'results_to_upload'
# client.load_table_from_dataframe(results_to_upload, ...)

In [ ]:
# --- 1. UPLOAD PREDICTIONS TO BIGQUERY ---
from google.cloud import bigquery

# NOTE: Replace 'your-gcp-project-id' with your actual Project ID!
PROJECT_ID = 'water-quality-param-pred'
client = bigquery.Client(project=PROJECT_ID)

# NOTE: Ensure the results_df and y_pred_gbr variables are still in scope from the previous step.
# If not, re-run the previous block that calculated y_pred_gbr and created results_df.

# Define the destination table
table_id = f"{PROJECT_ID}.water_data_raw.water_quality_predictions"

# Upload the DataFrame back to BigQuery
client.load_table_from_dataframe(
    results_to_upload,
    table_id,
    # Use WRITE_TRUNCATE to overwrite the table safely if it already exists
    job_config=bigquery.LoadJobConfig(write_disposition='WRITE_TRUNCATE')
).result()

print(f"\n✅ Successfully uploaded {len(results_to_upload)} predictions to BigQuery table: {table_id}")


✅ Successfully uploaded 434 predictions to BigQuery table: water-quality-param-pred.water_data_raw.water_quality_predictions
